In [ ]:
!pip install insightface onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1071492 sha256=e6466f8995525348254d145781e99caeb3edf1d3b8db5f84370bb41e1e187625
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


In [ ]:
import cv2
import shutil
import numpy as np
import os
import zipfile
import tempfile
import random
import gc
from pathlib import Path
from tqdm import tqdm
from insightface.app import FaceAnalysis
from google.colab import drive
from collections import defaultdict

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
ZIP_PATH = "/content/drive/MyDrive/Celeb-DF-v2.zip"
SAVE_BASE_PATH = "/content/drive/MyDrive/HECTO/Dataset/Celeb_frames"

NUM_FRAMES = 15
TARGET_SIZE = (256, 256)

In [ ]:
# providers=['CUDAExecutionProvider']로 GPU 강제 사용
detector = FaceAnalysis(allowed_modules=['detection', 'landmark_2d'], providers=['CUDAExecutionProvider'])
detector.prepare(ctx_id=0, det_size=(640, 640))

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 81789.80KB/s] 


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvid

In [ ]:
def get_hybrid_face(image, face_info, target_size=(256, 256)):
    try:
        h, w = image.shape[:2]
        bbox = face_info.bbox.astype(int)
        landmarks = getattr(face_info, 'kps', None)

        cx = (bbox[0] + bbox[2]) // 2
        cy = (bbox[1] + bbox[3]) // 2

        # 1️⃣ 정렬
        if landmarks is not None and len(landmarks) >= 2:
            left_eye, right_eye = landmarks[0], landmarks[1]
            dy, dx = right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]
            angle = np.degrees(np.arctan2(dy, dx))
            M = cv2.getRotationMatrix2D((float(cx), float(cy)), angle, 1.0)
            image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)

        # 2️⃣ 정사각 크롭
        side = int(max(bbox[2]-bbox[0], bbox[3]-bbox[1]) * 1.3)
        half = side // 2

        x1, y1 = cx - half, cy - half
        x2, y2 = cx + half, cy + half

        px1, py1 = max(0, -x1), max(0, -y1)
        px2, py2 = max(0, x2 - w), max(0, y2 - h)

        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)

        crop = image[y1:y2, x1:x2]

        if px1 or py1 or px2 or py2:
            crop = cv2.copyMakeBorder(
                crop, py1, py2, px1, px2,
                cv2.BORDER_CONSTANT, value=[0, 0, 0]
            )

        return cv2.resize(crop, target_size)

    except:
        return None

In [ ]:
def run_preprocessing():
    # 저장 경로 초기화
    if os.path.exists(SAVE_BASE_PATH):
        shutil.rmtree(SAVE_BASE_PATH)
    os.makedirs(SAVE_BASE_PATH, exist_ok=True)

    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        all_videos = [
            f for f in z.namelist()
            if f.lower().endswith('.mp4') and not os.path.basename(f).startswith('._')
        ]

        # ===============================
        # 🔧 데이터 필터링 및 샘플링
        # ===============================

        # 1️⃣ Celeb-real만 Real로 사용
        real_videos = [
            f for f in all_videos
            if 'celeb-real' in f.lower()
        ]

        # 2️⃣ Celeb-synthesis 전체
        fake_all = [
            f for f in all_videos
            if 'celeb-synthesis' in f.lower()
        ]
        # 3️⃣ Celeb-synthesis를 target ID 기준으로 미리 묶기 (속도 개선)
        fake_by_target = defaultdict(list)

        for fake_path in fake_all:
            fake_stem = Path(fake_path).stem   # 예: id0_id00_0001

            # ✅ target은 항상 뒤쪽
            if '_' not in fake_stem:
                continue

            target_id = fake_stem.split('_', 1)[1]  # id00_0001
            fake_by_target[target_id].append(fake_path)
        # 4️⃣ Celeb-real 기준 1:1 fake 매칭
        matched_real_videos = []
        matched_fake_videos = []
        used_fakes = set()

        for real_path in real_videos:
            target_id = Path(real_path).stem
            candidates = [
                f for f in fake_by_target.get(target_id, [])
                if f not in used_fakes
            ]

            if candidates:
                chosen = random.choice(candidates)
                matched_real_videos.append(real_path)
                matched_fake_videos.append(chosen)
                used_fakes.add(chosen)

        # ✅ 여기서 한 번만 덮어쓰기
        real_videos = matched_real_videos
        fake_videos = matched_fake_videos

        print(f"📊 REAL (matched): {len(real_videos)}, FAKE (matched): {len(fake_videos)}")
        assert len(real_videos) == len(fake_videos)

        # ===============================
        # 🔽 이후 전처리 로직 (기존 그대로)
        # ===============================

        for video_list, label_type in [(real_videos, "REAL"), (fake_videos, "FAKE")]:
            print(f"\n🔥 {label_type} 처리 중...")

            for file_path in tqdm(video_list):
                video_name = Path(file_path).stem
                label_dir = 'real' if label_type == "REAL" else 'fake'
                person_id = video_name.split('_')[0]

                save_dir = os.path.join(SAVE_BASE_PATH, label_dir, person_id, video_name)
                os.makedirs(save_dir, exist_ok=True)

                with tempfile.NamedTemporaryFile(suffix=".mp4") as tmp:
                    tmp.write(z.read(file_path))
                    tmp.flush()

                    cap = cv2.VideoCapture(tmp.name)
                    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    if total <= 0:
                        cap.release()
                        continue

                    indices = np.linspace(0, total - 1, NUM_FRAMES, dtype=int)

                    face_crops = []
                    frames_cache = []
                    last_face_info = None

                    for idx in indices:
                        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
                        ret, frame = cap.read()
                        if not ret or frame is None:
                            continue

                        frames_cache.append(frame)
                        faces = detector.get(frame)
                        crop = None

                        if faces:
                            face = max(
                                faces,
                                key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1])
                            )
                            crop = get_hybrid_face(frame, face)
                            if crop is not None:
                                last_face_info = face
                        elif last_face_info is not None:
                            crop = get_hybrid_face(frame, last_face_info)

                        if crop is not None:
                            if len(face_crops) == 0 and len(frames_cache) > 1:
                                for i in range(len(frames_cache) - 1):
                                    retro = get_hybrid_face(frames_cache[i], last_face_info)
                                    face_crops.append(retro if retro is not None else crop)
                            face_crops.append(crop)

                    cap.release()

                    if len(face_crops) == 0:
                        for i, frame in enumerate(frames_cache[:NUM_FRAMES]):
                            resized = cv2.resize(frame, TARGET_SIZE)
                            cv2.imwrite(os.path.join(save_dir, f"frame_{i}.jpg"), resized)
                    else:
                        while len(face_crops) < NUM_FRAMES:
                            face_crops.append(face_crops[-1])
                        for i in range(NUM_FRAMES):
                            cv2.imwrite(os.path.join(save_dir, f"frame_{i}.jpg"), face_crops[i])

                gc.collect()

    print(f"\n✅ 완료: {SAVE_BASE_PATH}")

run_preprocessing()

📊 REAL (matched): 558, FAKE (matched): 558

🔥 REAL 처리 중...


100%|██████████| 558/558 [09:32<00:00,  1.03s/it]



🔥 FAKE 처리 중...


100%|██████████| 558/558 [10:51<00:00,  1.17s/it]


✅ 완료: /content/drive/MyDrive/HECTO/Dataset/Celeb_frames


In [ ]:
SRC_DIR = "/content/drive/MyDrive/HECTO/Dataset/Celeb_frames"
DST_DIR = "/content/drive/MyDrive/HECTO/Dataset/final_files"
ZIP_NAME = "Celeb_frames_pp_final.zip"

os.makedirs(DST_DIR, exist_ok=True)

zip_path = os.path.join(DST_DIR, ZIP_NAME)

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(SRC_DIR):
        for file in files:
            file_path = os.path.join(root, file)

            # zip 내부 경로 (Celeb_frames/... 구조 유지)
            arcname = os.path.relpath(file_path, SRC_DIR)

            zipf.write(file_path, arcname)

print(f"✅ 압축 완료: {zip_path}")


✅ 압축 완료: /content/drive/MyDrive/HECTO/Dataset/final_files/Celeb_frames_pp_final.zip
